# BERT Review Sentiment

Testing out BERT by predicting the sentiment of ten thousand randomly sampled Steam reviews.

In [ ]:
# mount google drive and load csv file
from google.colab import drive
drive.mount('/content/drive')

